In [1]:
import math
import random
import os
import time
import json
from functools import partial
from collections import defaultdict

import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

### Check Version

In [2]:
tf.__version__

'2.9.1'

### GPU Configuration

In [3]:
gpus = tf.config.list_physical_devices('GPU')

2022-11-09 03:29:21.485844: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-09 03:29:21.492301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-09 03:29:21.492784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.experimental.set_memory_growth(gpus[0], True)

### Pandas Config

In [6]:
pd.set_option('display.max_column', None)

### Input Pipeline

In [7]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 4

In [8]:
CROP_WIDTH = 227
CROP_HEIGHT = 227

In [9]:
foldFiles = ["adience/unprocessed/fold_0_data.txt",
             "adience/unprocessed/fold_1_data.txt",
             "adience/unprocessed/fold_2_data.txt",
             "adience/unprocessed/fold_3_data.txt",
             "adience/unprocessed/fold_4_data.txt"]

In [10]:
genderMap = defaultdict(lambda : np.NaN)
genderMap['m'] = 0
genderMap['f'] = 1

In [11]:
ages = ['(0, 2)', '(4, 6)', '(8, 13)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
ageMap = defaultdict(lambda : np.NaN)
for i,a in enumerate(ages):
    ageMap[a] = i

### Dataset Generation

Dataset generation pipeline:
Input: 
- foldFile - Path to fold file. Fold file Describes which images are in the fold and its corresponding labels
- imgBaseFolder - Base folder to search image from
- imgPrefix - Prefix of image file
- genderMap - Map from ['m', 'f', 'u', None], to one hot index
- ageMap - Map from age category to one hot index
- imgWidth - Resulting image width
- imgHeigh - Resulting image height
- batchSize - Int or None, batch size
- configureDs - Function accepting dataset for performance configurations
- preBatch - List of (name, functions) pair that will be mapped before batching. name is used as name parameters for tf graph 
- postBatch - List of (name, functions) that will be mapped after batching. name is used as name parameters for tf graph

The processing functions should have signature function(img, label) -> (img, label)

Pipeline

Read Fold File -> Preprocess filename and labels (dataframe) -> Convert filename and labels to numpy array -> Convert filename and labels to tf dataset -> Parse images and labels -> Configure Dataset for performance -> Pre-Batching preprocessing -> Batch -> Post-Batching preprocessing -> Output

Some preprocessing steps can only be done before and some can only be done after batching, thats why there are seperated pre and post batching list

In [12]:
def generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, batchSize, configureDs=None, preBatch=[], postBatch=[]):
    
    def parseImage(filename):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img)
        img = tf.image.resize(img, [imgHeight, imgWidth])
         
        return img
    
    # Read Fold File
    foldData = []
    for f in foldFiles:
        foldData.append(pd.read_csv(f, sep="\t"))
    foldData = pd.concat(foldData)
    
    # Form File Name
    foldData['filename'] = foldData.apply(lambda r: os.path.join(imgBaseFolder, r['user_id'], f"{imgPrefix}.{r['face_id']}.{r['original_image']}"), axis=1)
    
    # Generate Label One Hot Index
    foldData['gender_ind'] = foldData['gender'].map(genderMap)
    foldData['age_ind'] = foldData['age'].map(ageMap)
    
    # Remove dirty data
    foldData.dropna(subset=['gender_ind', 'age_ind'], inplace=True)
    
    # Dataframe to numpy
    filenames = foldData['filename'].to_numpy()
    
    genderIndex = foldData['gender_ind'].to_numpy().astype(int)
    ageIndex = foldData['age_ind'].to_numpy().astype(int)
    
    # Numpy to Dataset
    fnDs = tf.data.Dataset.from_tensor_slices(filenames)
    genderIndDs = tf.data.Dataset.from_tensor_slices(genderIndex)
    ageIndDs = tf.data.Dataset.from_tensor_slices(ageIndex)
    
    # Parse Images
    imageDs = fnDs.map(parseImage, num_parallel_calls=tf.data.AUTOTUNE, name="parse_image")
    
    # Parse Labels
    genderLabDs = genderIndDs.map(lambda x: tf.one_hot(x, genderDepth), num_parallel_calls=tf.data.AUTOTUNE, name="gender_one_hot")
    ageLabDs = ageIndDs.map(lambda x: tf.one_hot(x, ageDepth), num_parallel_calls=tf.data.AUTOTUNE, name="age_one_hot")
    
    # Combine Labels
    labelDs = tf.data.Dataset.zip((genderLabDs, ageLabDs), name="label_zip")
    labelDs = labelDs.map(lambda g,a: {"gender": g, "age": a}, num_parallel_calls=tf.data.AUTOTUNE, name='label_dict')
    
    # Combine Images and Labels into dataset
    ds = tf.data.Dataset.zip((imageDs, labelDs))
    
    # Configure Performance
    if(configureDs is not None):
        ds = configureDs(ds)
    
    # Pre Batch Preprocessing
    for n,f in preBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    # Batch
    if(batchSize is not None):
        ds = ds.batch(batchSize, name="ds_batch")
    
    # Post Batch Preprocessing
    for n,f in postBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    return ds

### Preprocessings

In [13]:
def trainConfigPerformance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=100)
    ds = ds.repeat()
    return ds

In [14]:
def valConfigPerformance(ds):
    #ds = ds.cache()
    return ds

#### Preprocessing steps according to the reference paper

In [15]:
def trainPreprocessA(img, lab):
        
        # img = tf.image.random_crop(img, [CROP_HEIGHT, CROP_WIDTH, 3])
        
        ud = tf.random.uniform([], dtype=tf.float32)
        udCond = tf.less(ud, 0.5)
        
        img = tf.cond(udCond, lambda: tf.image.flip_up_down(img), lambda: img)
        
        lr = tf.random.uniform([], dtype=tf.float32)
        lrCond = tf.less(lr, 0.5)
        
        img = tf.cond(lrCond, lambda: tf.image.flip_left_right(img), lambda: img)
        
        img = tf.image.random_brightness(img, 63/255)
        
        img = tf.image.random_contrast(img, 0.2, 1.8) 
        
        return img, lab

In [16]:
def valPreprocessA(img, lab):
        
        img = tf.image.crop_to_bounding_box(img,
                                            (IMG_HEIGHT-CROP_HEIGHT) // 2,
                                            (IMG_WIDTH-CROP_WIDTH) // 2,
                                            CROP_HEIGHT,
                                            CROP_WIDTH)
        
        return img, lab

In [17]:
def imageStandardize(img, lab):
    
    img = tf.image.per_image_standardization(img)
    
    return img, lab

#### Label extractor 

In [18]:
def extractGenderLabel(img, lab):
    
    lab = lab['gender']
    
    return img, lab

In [19]:
def extractAgeLabel(img, lab):
    
    lab = lab['age']
    
    return img, lab

### Generate Folds for K-Folds validation

In [20]:
def generateFoldDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, valBatchSize, trainConfigureDs=None, trainPreBatch=[], trainPostBatch=[], valConfigureDs=None, valPreBatch=[], valPostBatch=[]):
    '''
        Returns list of (train, validation) datasets
    '''
    
    N = len(foldFiles)
    
    folds = []
    
    for i in range(N):
        
        trainFiles = foldFiles[:i]
        if(i < N-1):
            trainFiles.extend(foldFiles[i+1:])
            
        valFiles = foldFiles[i]
        
        trainDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, configureDs=trainConfigureDs, preBatch=trainPreBatch, postBatch=trainPostBatch)
        valDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, valBatchSize, configureDs=valConfigureDs, preBatch=valPreBatch, postBatch=valPostBatch)
        
        folds.append((trainDs, valDs))
        
    return folds

In [21]:
trainPreBatch = [("train_process_a", trainPreprocessA), 
                 ("train_standardize", imageStandardize), 
                 ("train_extract_gender", extractGenderLabel)]

valPreBatch = [("val_standardize", imageStandardize), 
               ("val_extract_gender", extractGenderLabel)]

In [22]:
folds = generateFoldDs(foldFiles, "adience/faces", "coarse_tilt_aligned_face", genderMap, ageMap, 2, 8, 
                       IMG_WIDTH, IMG_HEIGHT, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, 
                       trainConfigureDs=trainConfigPerformance, trainPreBatch=trainPreBatch, 
                       valConfigureDs=valConfigPerformance, valPreBatch=valPreBatch)

2022-11-09 03:29:21.741661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 03:29:21.742310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-09 03:29:21.742575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-09 03:29:21.742820: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [23]:
len(folds)

5

### Check data

In [24]:
for x,y in folds[0][0].take(1):
    pass

In [25]:
x

<tf.Tensor: shape=(4, 256, 256, 3), dtype=float32, numpy=
array([[[[-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         ...,
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ]],

        [[-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         ...,
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ]],

        [[-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         ...,
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267 ],
         [-1.2274075 , -1.3971285 , -1.2813267

In [26]:
y

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)>

### Model

In [27]:
def createModel():
    inp = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    conv1 = tf.keras.layers.Conv2D(96, 7, strides=(4,4), activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv1')
    pool1 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool1')

    conv2 = tf.keras.layers.Conv2D(256, 5, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv2')
    pool2 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool2')

    conv3 = tf.keras.layers.Conv2D(384, 3, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv3')
    pool3 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool3')

    flat1 = tf.keras.layers.Flatten(name='flat1')
    fc1 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc1')
    do1 = tf.keras.layers.Dropout(0.5, name='do1')

    fc2 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc2')
    do2 = tf.keras.layers.Dropout(0.5, name='do2')

    fc3 = tf.keras.layers.Dense(2, activation='softmax', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='fc3')

    o = conv1(inp)
    o = pool1(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [2, 2], [2, 2], [0, 0]])
    o = conv2(o)
    o = pool2(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [1, 1], [1, 1], [0, 0]])
    o = conv3(o)
    o = pool3(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = flat1(o)
    o = fc1(o)
    o = do1(o)


    o = fc2(o)
    o = do2(o)

    o = fc3(o)

    model = tf.keras.models.Model(inputs=inp, outputs=o, name='cnn_6')
    
    return model

### Compile Model

In [28]:
STEPS_PER_EPOCH = 1000

EPOCH = 300

START_EPOCH = 0
END_EPOCH = START_EPOCH + EPOCH

#### Learning Rate Schedule

In [29]:
def schedule(epoch, lr):
    if(epoch == 10000*50//STEPS_PER_EPOCH):
        return lr * 0.1
    else:
        return lr

In [30]:
logBasePath = "log/CNN_6"
logPrefix = "log"

In [31]:
for i, (train, val) in enumerate([folds[0]]):
    
    model = createModel()
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    
    tboardCb = tf.keras.callbacks.TensorBoard(log_dir=f"{logBasePath}/{logPrefix}_{i}", histogram_freq=1, profile_batch=(2,10))
    lrSchedule = tf.keras.callbacks.LearningRateScheduler(schedule)
    
    cbs = [tboardCb]
    
    model.evaluate(val)
    model.fit(train, initial_epoch=START_EPOCH, epochs=END_EPOCH, steps_per_epoch=STEPS_PER_EPOCH, callbacks=cbs, validation_data=val)

2022-11-09 03:29:28.455759: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-09 03:29:28.455778: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-09 03:29:28.455796: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2022-11-09 03:29:28.566946: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-11-09 03:29:28.568345: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2022-11-09 03:29:29.120992: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  52/3528 [..............................] - ETA: 10s - loss: 0.6846 - accuracy: 0.6202  

2022-11-09 03:29:29.906417: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3528/3528 [==============================] - 10s 3ms/step - loss: 0.6957 - accuracy: 0.4768
Epoch 1/300
   1/1000 [..............................] - ETA: 7:45 - loss: 0.6534 - accuracy: 0.7500

2022-11-09 03:29:39.286868: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-09 03:29:39.286888: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.


...

1000/1000 [==============================] - 14s 14ms/step - loss: 0.4741 - accuracy: 0.7638 - val_loss: 0.4705 - val_accuracy: 0.7743
Epoch 61/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4105 - accuracy: 0.8198 - val_loss: 0.4647 - val_accuracy: 0.7821
Epoch 62/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4195 - accuracy: 0.8177 - val_loss: 0.4885 - val_accuracy: 0.7572
Epoch 63/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4593 - accuracy: 0.7810 - val_loss: 0.5448 - val_accuracy: 0.7111
Epoch 64/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3909 - accuracy: 0.8270 - val_loss: 0.4846 - val_accuracy: 0.7658
Epoch 65/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3922 - accuracy: 0.8357 - val_loss: 0.4667 - val_accuracy: 0.7982
Epoch 66/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4029 - accuracy: 0.8240 - val_loss: 0.4586 

1000/1000 [==============================] - 14s 14ms/step - loss: 0.2435 - accuracy: 0.9087 - val_loss: 0.3109 - val_accuracy: 0.8870
Epoch 116/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2637 - accuracy: 0.8913 - val_loss: 0.4206 - val_accuracy: 0.8460
Epoch 117/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2064 - accuracy: 0.9222 - val_loss: 0.3168 - val_accuracy: 0.8721
Epoch 118/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2295 - accuracy: 0.9075 - val_loss: 0.3832 - val_accuracy: 0.8255
Epoch 119/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2248 - accuracy: 0.9160 - val_loss: 0.2999 - val_accuracy: 0.8780
Epoch 120/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2606 - accuracy: 0.8913 - val_loss: 0.2404 - val_accuracy: 0.9024
Epoch 121/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2009 - accuracy: 0.9235 - val_loss: 0

1000/1000 [==============================] - 14s 14ms/step - loss: 0.1262 - accuracy: 0.9528 - val_loss: 0.2152 - val_accuracy: 0.9271
Epoch 171/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1798 - accuracy: 0.9355 - val_loss: 0.2155 - val_accuracy: 0.9175
Epoch 172/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1711 - accuracy: 0.9312 - val_loss: 0.2086 - val_accuracy: 0.9317
Epoch 173/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1489 - accuracy: 0.9450 - val_loss: 0.2683 - val_accuracy: 0.9089
Epoch 174/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1405 - accuracy: 0.9488 - val_loss: 0.1863 - val_accuracy: 0.9282
Epoch 175/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1528 - accuracy: 0.9398 - val_loss: 0.4570 - val_accuracy: 0.8279
Epoch 176/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1789 - accuracy: 0.9310 - val_loss: 0

1000/1000 [==============================] - 14s 14ms/step - loss: 0.1398 - accuracy: 0.9460 - val_loss: 0.1743 - val_accuracy: 0.9343
Epoch 226/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1255 - accuracy: 0.9538 - val_loss: 0.1876 - val_accuracy: 0.9475
Epoch 227/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1464 - accuracy: 0.9490 - val_loss: 0.2083 - val_accuracy: 0.9211
Epoch 228/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1299 - accuracy: 0.9530 - val_loss: 0.1866 - val_accuracy: 0.9391
Epoch 229/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1441 - accuracy: 0.9473 - val_loss: 0.1574 - val_accuracy: 0.9529
Epoch 230/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0993 - accuracy: 0.9657 - val_loss: 0.1919 - val_accuracy: 0.9309
Epoch 231/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1398 - accuracy: 0.9525 - val_loss: 0

1000/1000 [==============================] - 14s 14ms/step - loss: 0.1342 - accuracy: 0.9513 - val_loss: 0.3641 - val_accuracy: 0.8727
Epoch 281/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1051 - accuracy: 0.9605 - val_loss: 0.1521 - val_accuracy: 0.9472
Epoch 282/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1448 - accuracy: 0.9477 - val_loss: 0.1134 - val_accuracy: 0.9633
Epoch 283/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1043 - accuracy: 0.9617 - val_loss: 0.2446 - val_accuracy: 0.9053
Epoch 284/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1495 - accuracy: 0.9488 - val_loss: 0.1780 - val_accuracy: 0.9447
Epoch 285/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1391 - accuracy: 0.9465 - val_loss: 0.1491 - val_accuracy: 0.9455
Epoch 286/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1039 - accuracy: 0.9663 - val_loss: 0